In [1]:
import numpy as np
import pandas as pd
import os

In [3]:
train_articles = []

for article in os.listdir('data/train/'):
    for file in os.listdir(f'data/train/{article}'):
        with open(f'data/train/{article}/{file}', 'r', encoding='utf-8') as file:
            train_articles.append(file.read())

In [5]:
len(train_articles)

190

In [6]:
train_labels = pd.read_csv('data/train.csv')

In [10]:
train_labels.shape

(95, 2)

In [11]:
train_labels.head()

,id,real_text_id
0,0,1
1,1,2
2,2,1
3,3,2
4,4,2


In [61]:
train_file_1 = []
train_file_2 = []
train_target = []

for article in os.listdir('data/train/'):
    for file_name in os.listdir(f'data/train/{article}'):
        file_path = f'data/train/{article}/{file_name}'
        with open(file_path, 'r', encoding='utf-8') as file:
            if file_name == 'file_1.txt':
                train_file_1.append(file.read())
            else:
                train_file_2.append(file.read())

train_target = train_labels.real_text_id.values

In [63]:
train_df = pd.DataFrame(data = list(zip(train_file_1, train_file_2, train_target)), columns = ['file_1', 'file_2', 'target'])

In [64]:
train_df

,file_1,file_2,target
0,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...,1
1,China\nThe goal of this project involves achie...,The project aims to achieve an accuracy level ...,2
2,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...,1
3,China\nThe study suggests that multiple star s...,The importance for understanding how stars evo...,2
4,Dinosaur Rex was excited about his new toy set...,Analyzing how fast stars rotate within a galax...,2
...,...,...,...
90,A main focus of modern cosmology is to underst...,A key focus of modern cosmology is to understa...,2
91,"APEX, as its name suggests, serves as a guide ...","APEX, as its name suggests, serves as a guide ...",1
92,FORS1 and FORS2 are early instruments of the V...,FORS1 and FORS2 are early instruments of the V...,2
93,The observations of the Pluto-Charon system an...,The observations of the Pluto-Charon binary an...,2


In [76]:
comb_train_df = train_df['file_1'] + " " + train_df['file_2']

In [89]:
test_file_1 = []
test_file_2 = []

for article in os.listdir('data/test/'):
    for file_name in os.listdir(f'data/test/{article}'):
        file_path = f'data/test/{article}/{file_name}'
        with open(file_path, 'r', encoding='utf-8') as file:
            if file_name == 'file_1.txt':
                test_file_1.append(file.read())
            else:
                test_file_2.append(file.read())


In [90]:
test_df = pd.DataFrame(data = list(zip(test_file_1, test_file_2)), columns = ['file_1', 'file_2'])

In [91]:
test_df

,file_1,file_2
0,"""Music"" Music music music Music music Music mu...",Since its launch on Paranal observatory's Very...
1,underground exploration on SN's birth has prov...,SN 1987A provides valuable insights as newer o...
2,This research aimed to understand how star sha...,ChromeDriver music player\nThis study focused ...
3,Using OmegaCAM's wide field capabilities spann...,"greek translation :\nvazhi (megaCAM), territor..."
4,AssemblyCulture AssemblyCulture AssemblyCultur...,XClass is software tool that helps astronomers...
...,...,...
1063,Alongside the detailed studies mentioned earli...,Alongside the detailed studies mentioned earli...
1064,"At this meeting, we gained a new outlook on th...","At this meeting, we gained a new outlook on th..."
1065,ESO Reflex is designed to handle essential tas...,ESO Reflex is designed to supply essential com...
1066,Even greater angular resolution is possible wi...,Higher angular resolution can also be achieved...


In [92]:
comb_test_df = test_df['file_1'] + " " + test_df['file_2']

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
tfidf = TfidfVectorizer()

In [56]:
from sklearn.pipeline import Pipeline

In [57]:
from sklearn.model_selection import GridSearchCV

In [58]:
from sklearn.ensemble import RandomForestClassifier

In [104]:
model = RandomForestClassifier()

In [117]:
params = {
    'model__criterion': ['gini', 'entropy', 'log_loss'],
    'model__max_depth': [2, 4, 6, 8, 10, 12, 14, 16]
}

In [118]:
pipe = Pipeline([('tfidf', tfidf), ('model', model)])

In [119]:
final_model = GridSearchCV(pipe, param_grid = params, cv = 3, verbose = 1)

In [120]:
X = comb_train_df

In [121]:
Y = train_df.target

In [122]:
final_model.fit(X, Y)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('model', RandomForestClassifier())]),
             param_grid={'model__criterion': ['gini', 'entropy', 'log_loss'],
                         'model__max_depth': [2, 4, 6, 8, 10, 12, 14, 16]},
             verbose=1)

In [123]:
prediction = final_model.predict(comb_test_df)

In [124]:
len(comb_test_df)

1068

In [125]:
sub_df = pd.DataFrame(data = list(zip([i for i in range(0, len(comb_test_df))], prediction)), columns = ['id', 'real_text_id'])

In [126]:
sub_df

,id,real_text_id
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
1063,1063,1
1064,1064,2
1065,1065,2
1066,1066,2


In [127]:
sub_df.to_csv('sub2.csv', index=False, index_label=False)